<a href="https://colab.research.google.com/github/andrjohns/colab-testing/blob/main/dummy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bayesian Workflow - Probabilistic AI School 2023

## Preparation

In this tutorial we will be using the [cmdstanr](https://mc-stan.org/cmdstanr/articles/cmdstanr.html) R interface to CmdStan. The installation of the R dependencies and building of CmdStan can take some time in CoLab, so we'll download some pre-built versions of these.

In [ ]:
# Download the pre-built libraries and other files for the tutorial
system("git clone https://github.com/andrjohns/ProbAI-2023")
setwd("ProbAI-2023")

# Unpack CmdStan
system("tar -zxf cmdstan.tar.gz")

# Add the pre-built libraries to the current R session
.libPaths("colab-libs")

In addition to cmdstanr, we'll be using the [bayesplot](https://mc-stan.org/bayesplot/articles/graphical-ppcs.html) package for our graphical model checking, so let's load that library as well:

In [ ]:
library(cmdstanr)
library(bayesplot)
library(ggplot2)
set_cmdstan_path("cmdstan-2.32.2")

## Epilepsy RCT Workflow
### Data
Let's load our dataset and look at the general structure:


In [ ]:
load("epilepsy.RData")
head(epilepsy_rct)

### Initial Model

We've decided to use a Poisson Generalised Linear Model with a log-link as our initial attempt for modelling the data:

$$
y_i \sim Poisson(\lambda_i) \\
\lambda_i = \exp(\alpha + x_i^T\beta) \\
\alpha \sim N(0,5) \\
\beta_{1:4} \sim N(0,1)
$$

Let's have a look at how we would specify this in Stan:

In [ ]:
cat(readLines("Stan/poisson.stan"), sep = "\n")

Now that we have our model, and we've defined the data we'll need, let's structure our epilepsy observations to the right format:

In [ ]:
epilepsy_stan <- list(
  N = length(unique(epilepsy_rct$patient)),
  T = length(unique(epilepsy_rct$visit)),
  K = 4,
  ID = epilepsy_rct$patient,
  x = epilepsy_rct[,c("treatment","age","baseline","base_x_treat")],
  seizures = epilepsy_rct$seizures
)

Now we're ready to fit our model! Remember that Stan is a *compiled* language, so first we need to compile our Stan model into an executable:

In [ ]:
mod <- cmdstan_model("Stan/poisson.stan")

Now that it's compiled, we can begin the sampling process:

In [ ]:
fit <- mod$sample(
  data = epilepsy_stan,
  parallel_chains = 4,
  refresh = 0,
  show_messages = FALSE,
  show_exceptions = FALSE,
  seed = 2023
)

Let's check our diagnostics to see how the sampling went. First up, do the traceplots indicate that the chains have converged?

In [ ]:
mcmc_trace(fit$draws("beta"))

Looking good! Let's also check our R-hat statistic and effective sample sizes:

In [ ]:
fit$summary("beta")

Not bad! Let's have a look at our posterior-predictive checks:

In [ ]:
ppc_dens_overlay(y = epilepsy_stan$seizures,
                 fit$draws("ypred", format = "draws_matrix")[1:20,]) +
  coord_cartesian(xlim=c(0,100))

It looks like there are some areas where our model doesn't represent our data very well, let's see what we can do about that.

### Random-Effects Model

We'll add a random intercept for each individual, to relax the assumption of equal mean and variance in the Poisson:

$$
y_i \sim Poisson(\lambda_i) \\
\lambda_i = \exp(\alpha + x_i^T\beta + u_i) \\
\alpha \sim N(0,5) \\
\beta_{1:4} \sim N(0,1) \\
u_i \sim N(0,\sigma) \\
\sigma \sim Cauchy^+(0,5)
$$

How does this look in our Stan model?

In [ ]:
cat(readLines("Stan/poisson_ranef.stan"), sep = "\n")

Now let's follow the same process of compiling our model and then running sampling:

In [ ]:
mod_ranef <- cmdstan_model("Stan/poisson_ranef.stan")
fit_ranef <- mod_ranef$sample(
  data = epilepsy_stan,
  parallel_chains = 4,
  refresh = 0,
  show_messages = FALSE,
  show_exceptions = FALSE,
  seed = 2023
)

How do our convergence diagnostics and effective sample sizes look?

In [ ]:
mcmc_trace(fit_ranef$draws("beta"))
fit_ranef$summary("beta")

How about our fit to the data?

In [ ]:
ppc_dens_overlay(y = epilepsy_stan$seizures,
                 fit_ranef$draws("ypred", format = "draws_matrix")[1:20,]) +
  coord_cartesian(xlim=c(0,100))

Much better! Now that we have a possible model, let's look at making it a little more efficient

### Marginalisation

As a first step in our marginalisation journey, let's change our normally-distributed random effect to a Gamma with equal shape and rate parameters:

$$
y_i \sim Poisson(\lambda_i\theta_i) \\
\lambda_i = \exp(\alpha + x_i^T\beta) \\
\alpha \sim N(0,5) \\
\beta_{1:4} \sim N(0,1) \\
\theta_i \sim Gamma(\phi,\phi) \\
\phi \sim Cauchy^+(0,5)
$$

Which we would represent in Stan using:

In [ ]:
cat(readLines("Stan/poisson_gamma.stan"), sep = "\n")

Let's fit our new model and check our posterior-predictives:

In [ ]:
mod_gamma <- cmdstan_model("Stan/poisson_gamma.stan")
fit_gamma <- mod_gamma$sample(
  data = epilepsy_stan,
  parallel_chains = 4,
  refresh = 0,
  show_messages = FALSE,
  show_exceptions = FALSE,
  seed = 2023
)

In [ ]:
ppc_dens_overlay(y = epilepsy_stan$seizures,
                 fit_gamma$draws("ypred", format = "draws_matrix")[1:20,]) +
  coord_cartesian(xlim=c(0,100))

Still looking good! But what's the point? 

Remember that we can represent the Poisson with a Gamma-distributed random effect as a Negative-Binomial parameterised by its mean and dispersion:

$$
\int Poisson(y | \lambda\theta) \cdot Gamma(\theta | \phi, \phi) d\theta = NB(y|\lambda, \phi)
$$

But don't just take my word for it, let's verify this in R by comparing the numerically integrated Poisson-Gamma with the Negative-Binomial:

In [ ]:
lambda <- 2.65
y <- 4
phi <- 1.5

poisson_gamma_pdf <- function(theta, y, lambda, phi) {
  exp(dpois(y, lambda * theta, log = TRUE) + dgamma(theta, shape = phi, rate = phi, log = TRUE))
}

integrate(poisson_gamma_pdf, 0, Inf, y, lambda, phi)
dnbinom(y, mu = lambda, size = phi)

Brilliant! Let's put this into practice with Stan:

In [ ]:
cat(readLines("Stan/nb.stan"), sep = "\n")

In [ ]:
mod_nb <- cmdstan_model("Stan/nb.stan")
fit_nb <- mod_nb$sample(
  data = epilepsy_stan,
  parallel_chains = 4,
  refresh = 0,
  show_messages = FALSE,
  show_exceptions = FALSE,
  seed = 2023
)

In [ ]:
ppc_dens_overlay(y = epilepsy_stan$seizures,
                 fit_nb$draws("ypred", format = "draws_matrix")[1:20,]) +
  coord_cartesian(xlim=c(0,100))

Looking good again! How are our sampling runtimes looking?

In [ ]:
fit_ranef$time()$total
fit_gamma$time()$total
fit_nb$time()$total

Well that's a pretty impressive improvement! How much better can we do if we use the optimised GLM distributions in Stan?

In [ ]:
cat(readLines("Stan/nb_glm.stan"), sep = "\n")

In [ ]:
mod_nb_glm <- cmdstan_model("Stan/nb_glm.stan")
fit_nb_glm <- mod_nb_glm$sample(
  data = epilepsy_stan,
  parallel_chains = 4,
  refresh = 0,
  show_messages = FALSE,
  show_exceptions = FALSE,
  seed = 2023
)

In [ ]:
ppc_dens_overlay(y = epilepsy_stan$seizures,
                 fit_nb_glm$draws("ypred", format = "draws_matrix")[1:20,]) +
  coord_cartesian(xlim=c(0,100))

In [ ]:
fit_ranef$time()$total
fit_gamma$time()$total
fit_nb$time()$total
fit_nb_glm$time()$total

Now that's a nice (and scalable) improvement!

### Compare Models

Now that we've finished developing our models (for now), how do they differ? Why would we prefer one over the other? Let's look at our inferences for the treatment effect:

In [ ]:
fit_ranef$summary("beta[1]")
fit_nb_glm$summary("beta[1]")

The more efficient sampling of the NG-GLM model resulted in less (computational) uncertainty in our estimates, a narrower posterior and greater effective sample size.

We can also see this by plotting the treatment effect posterior for each model:

In [ ]:
mcmc_dens(fit_ranef$draws("beta[1]")) + coord_cartesian(xlim=c(-1,0.5))
mcmc_dens(fit_nb_glm$draws("beta[1]")) + coord_cartesian(xlim=c(-1,0.5))